In [381]:
import numpy as np 
import torch 
import pandas as pd
import yfinance as yf
from arch import arch_model
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.optimize import minimize

In [382]:
data = yf.download('^GSPC', start="2015-01-01", end="2025-01-01")

[*********************100%***********************]  1 of 1 completed


In [383]:
data.reset_index(inplace=True)

In [384]:
data = data.Close

In [385]:
data.reset_index(inplace=True)

In [386]:
data['log_returns'] = np.log(data['^GSPC']/data['^GSPC'].shift(-1)) #non-squared 
data['volatility'] = data['log_returns'].rolling(window=5).apply(lambda x: (np.sqrt(np.sum(x**2)))) #non-sduared volatility
data['volatility'] = data['volatility']*100
data['log_returns'] = data['log_returns']*100
data.drop(['^GSPC','index'], axis=1, inplace=True)
data.fillna(0, inplace=True)

In [387]:
train_len = int(len(data) * 0.8)
val_len = int(len(data)*0.1 + train_len)
test_len = int(len(data)-train_len-val_len)

In [388]:
train_data = data.iloc[:train_len]
val_data = data.iloc[train_len:val_len]
test_data = data.iloc[val_len:int(len(data))]

In [389]:
train_data

Ticker,log_returns,volatility
0,1.844721,0.000000
1,0.893325,0.000000
2,-1.156274,0.000000
3,-1.773017,0.000000
4,0.843932,3.064932
...,...,...
2007,1.455714,2.526099
2008,-0.585095,2.338602
2009,0.405784,2.194310
2010,1.209347,2.503352


In [390]:
garch = arch_model(train_data['log_returns'], vol='GARCH', p=1,q=1, mean='Zero')
garch_fit = garch.fit()

Iteration:      1,   Func. Count:      5,   Neg. LLF: 11170.316193628138
Iteration:      2,   Func. Count:     15,   Neg. LLF: 4643.98763551963
Iteration:      3,   Func. Count:     22,   Neg. LLF: 5106.335316298678
Iteration:      4,   Func. Count:     28,   Neg. LLF: 2635.042239531366
Iteration:      5,   Func. Count:     32,   Neg. LLF: 2635.042094583123
Iteration:      6,   Func. Count:     35,   Neg. LLF: 2635.0420945838796
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2635.042094583123
            Iterations: 6
            Function evaluations: 35
            Gradient evaluations: 6


In [391]:
garch_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                       Zero Mean - GARCH Model Results                        
==============================================================================
Dep. Variable:            log_returns   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2635.04
Distribution:                  Normal   AIC:                           5276.08
Method:            Maximum Likelihood   BIC:                           5292.90
                                        No. Observations:                 2012
Date:                Tue, Mar 18 2025   Df Residuals:                     2012
Time:                        14:46:43   Df Model:                            0
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0385  1.101e-02      3.497  4.697e-04 [1.693e-02,6.010e-02]
alpha[1]       0.1960  3.265e-02      6.001  1.957e-09     [  0.132,  0.260]
beta[1]        0.7795  3.050e-02     25.556 4.740e-144     [  0.720,  0.839]
============================================================================

Covariance estimator: robust
"""

In [392]:


def generate_ground_garch(omega, alpha, beta, n=1000):
    """
    Generates synthetic GARCH(1,1) data.
    Returns residuals (ϵ_t) and volatility (σ_t²).
    """
   

    am = arch_model(None, mean='Zero', vol='GARCH', p=1, q=1)
    params = np.array([omega, alpha, beta])
    am_data = am.simulate(params, n)

    return am_data['data'].shift(-1).to_numpy(), am_data['volatility'].shift(-1).to_numpy()

class GARCHDataset(Dataset):
    """
    Custom PyTorch Dataset for GARCH(1,1) data.
    Each sample consists of:
    - Input: (1, ϵ²_{t-1}, σ²_{t-1})
    - Target: σ²_t
    """
    def __init__(self, residuals, volatility):
        """
        residuals: Array of residuals (ϵ_t)
        volatility: Array of volatility (σ_t²)
        """
        self.residuals = residuals
        self.volatility = volatility
        valid_indices = ~np.isnan(residuals) & ~np.isnan(volatility)
        residuals = residuals[valid_indices]
        volatility = volatility[valid_indices]
        # Create input vectors (1, ϵ²_{t-1}, σ²_{t-1})
        self.inputs = np.column_stack([
            np.ones_like(residuals),  # 1
            np.square(np.roll(residuals, 1)),  # ϵ²_{t-1}
            np.square(np.roll(volatility, 1))  # σ²_{t-1}
        ])

        # Remove the first sample (t=0) because it has no t-1
        self.inputs = self.inputs[1:]
        self.targets = np.square(volatility[1:])

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        """
        Returns:
        - Input: (1, ϵ²_{t-1}, σ²_{t-1}) as a tensor
        - Target: σ²_t as a tensor
        """
        input_sample = torch.tensor(self.inputs[idx], dtype=torch.float32)
        target_sample = torch.tensor(self.targets[idx], dtype=torch.float32)
        return input_sample, target_sample

# Generate synthetic GARCH(1,1) data
residuals, volatility = generate_ground_garch(omega=0.1, alpha=0.2, beta=0.7, n=1000)

# Create the dataset
dataset = GARCHDataset(residuals, volatility)

# Create a DataLoader for batching
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Example: Iterate through the DataLoader
for batch_inputs, batch_targets in dataloader:
    print("Batch Inputs:", batch_inputs)
    print("Batch Targets:", batch_targets)
    break

Batch Inputs: tensor([[1.0000e+00, 5.6845e+00, 1.0019e+00],
        [1.0000e+00, 4.0256e-01, 1.9382e+00],
        [1.0000e+00, 4.3501e+00, 1.5373e+00],
        [1.0000e+00, 6.5547e+00, 2.0461e+00],
        [1.0000e+00, 2.9791e-01, 2.8432e+00],
        [1.0000e+00, 4.0859e+00, 2.1498e+00],
        [1.0000e+00, 2.7027e-01, 2.4221e+00],
        [1.0000e+00, 2.6619e+00, 1.8495e+00],
        [1.0000e+00, 1.4847e+00, 1.9270e+00],
        [1.0000e+00, 2.7053e+00, 1.7459e+00],
        [1.0000e+00, 1.3921e+00, 1.8632e+00],
        [1.0000e+00, 1.3537e+00, 1.6826e+00],
        [1.0000e+00, 1.6531e+00, 1.5486e+00],
        [1.0000e+00, 2.8131e+00, 1.5146e+00],
        [1.0000e+00, 3.0418e-01, 1.7229e+00],
        [1.0000e+00, 2.6775e-03, 1.3668e+00],
        [1.0000e+00, 1.3727e-01, 1.0573e+00],
        [1.0000e+00, 1.1068e+00, 8.6758e-01],
        [1.0000e+00, 1.1627e-02, 9.2867e-01],
        [1.0000e+00, 9.0321e-01, 7.5239e-01],
        [1.0000e+00, 1.3719e+00, 8.0732e-01],
        [1.0000e+00,

In [393]:
def generate_ground_gjrgarch(omega, alpha, lmbda, beta, n = 1000):
    am = arch_model(None, mean='Zero', p =1, q = 1, o =1)
    params = np.array([omega,alpha,lmbda,beta])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()

In [394]:
def generate_ground_figarch(omega, beta, phi ,d, n = 1000):
    am = arch_model(None, mean='Zero', vol='FIGARCH')
    params= np.array([omega, beta, phi, d])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()

In [395]:
generate_ground_garch(0.5,0.2,0.1);

In [396]:
generate_ground_gjrgarch(0.1,0.2,0.3,0.4);

In [397]:
generate_ground_figarch(0.1,0.2,0.3,0.4);

In [398]:
class RNNGARCH (torch.nn.Module): 
    def __init__(self,  input_size = 1, hidden_size =1):
        super(RNNGARCH, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.linear = nn.Linear(3, 1, bias=False)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        sigma_t = self.linear(x)
        return sigma_t.squeeze()
    
    
    def get_garch_parameters(self) -> tuple[float, float, float]:
        """
        Returns the GARCH(1,1) parameters (ω, α, β) from the layer's weights.
        """
        weights = self.linear.weight.data.squeeze().tolist()
        omega, alpha, beta = weights
        return omega, alpha, beta

        


In [399]:
# Check for NaNs in the dataset
for batch_inputs, batch_targets in dataloader:
    if torch.isnan(batch_targets).any():
        print("NaNs found in inputs or targets!")
    if torch.isinf(batch_inputs).any() or torch.isinf(batch_targets).any():
        print("Infs found in inputs or targets!")

In [400]:
class GARCHNegativeLogLikelihood(nn.Module):
   
    def __init__(self):
        super().__init__()

    def forward(self, residuals: torch.Tensor, estimated_volatility: torch.Tensor) -> torch.Tensor:
        
        # Ensure estimated_volatility is positive to avoid numerical issues
        estimated_volatility = torch.clamp(estimated_volatility, min=1e-8)

        # Compute the two terms of the negative log-likelihood
        term1 = 0.5 * torch.log(estimated_volatility)  # log(σ_t²) / 2
        term2 = (residuals ** 2) / (2 * estimated_volatility)  # ϵ_t² / (2 * σ_t²)

        # Sum the terms and average over the batch
        loss = torch.mean((term1 + term2))
        return loss

In [401]:
# Create the GARCH(1,1) RNN layer
garch_layer = RNNGARCH()

# Create the loss function
loss_fn = nn.MSELoss()

# Create an optimizer
optimizer = torch.optim.Adam(garch_layer.parameters(), lr=0.01)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_inputs, batch_targets in dataloader:
        # Forward pass
        estimated_volatility = garch_layer(batch_inputs)

        # Compute loss
        residuals = batch_targets  # Assuming batch_targets are σ²_t, residuals are ϵ_t = sqrt(σ²_t)
        loss = loss_fn(residuals, estimated_volatility)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Print average loss for the epoch
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch + 1}, Average Loss: {epoch_loss / len(dataloader)}")

# Get the trained GARCH(1,1) parameters
omega, alpha, beta = garch_layer.get_garch_parameters()
print(f"Trained GARCH(1,1) Parameters: ω={omega}, α={alpha}, β={beta}")

Epoch 100, Average Loss: 1.6128880634062063e-14
Epoch 200, Average Loss: 2.6999004876580954e-13
Epoch 300, Average Loss: 3.335943153659482e-11
Epoch 400, Average Loss: 6.402443415254311e-05
Epoch 500, Average Loss: 0.00013874447361672537
Epoch 600, Average Loss: 6.210604173041007e-08
Epoch 700, Average Loss: 2.12940318572441e-08
Epoch 800, Average Loss: 4.796076114033099e-06
Epoch 900, Average Loss: 4.806647355692917e-05
Epoch 1000, Average Loss: 1.1889066849068985e-08
Trained GARCH(1,1) Parameters: ω=0.09993383288383484, α=0.20000706613063812, β=0.7001566886901855
